In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import rebound as rb

In [2]:
tup_num = 7
e_b = np.linspace(0, 0.7, tup_num)
a_p = np.linspace(1, 5, tup_num)
mass = list(np.linspace (0.1, 0.9, tup_num))
Np = 5

tup_list = []

for e in e_b:
    for a in a_p:
        tup_list.append((e,a,Np))

In [12]:
mass

[0.1,
 0.23333333333333334,
 0.3666666666666667,
 0.5,
 0.6333333333333333,
 0.7666666666666666,
 0.9]

In [19]:
def survival(initial):    
    (eb, ap, Np) = initial

    sim = rb.Simulation()
    sim.integrator = "whfast"
    
    m1 = 1
    mu = ratio
    m2 = abs((m1*mu)/(1-mu))
    sim.add(m=m1, hash="Binary 1")
    sim.add(m=m2, a=1, e= eb, hash="Binary 2")
    
    #initializing Np massless planets
    
    for p in range(Np):
        f_plan = np.random.rand()*2*np.pi
        sim.add(m=0, a= ap, e=0, f= f_plan)
    
    #array to keep track of survival time
    sim.move_to_com()
    
    #integrate
    N_times = int(100)
    N_orbit = (1e4)*2*np.pi
    times = np.linspace(0,N_orbit,N_times)
    surv = np.zeros(Np)
    
    for i, time in enumerate(times):
            sim.integrate(time, exact_finish_time=0)
            
            for num in reversed(range(2, sim.N)):
                p = sim.particles[num]

                if (p.x**2 + p.y**2) > (100)**2:
                    surv[num-2] = time
                    print(f'removing planet {num}')
                    sim.remove(num)

            if sim.N==2:
                break
    surv[(surv==0)] = time
    
    print(f'simulation finished, {len(sim.particles)-2} planets remaining')
    return np.mean(surv)

In [20]:
def mass_ratio(m):
    ratio_list = []
    
    for ratio in m:
        pool = rb.InterruptiblePool(processes=16)
        mapping = pool.map(func= survival, iterable= tup_list)
    
        ratio_list.append(mapping)
    
    return ratio_list

In [21]:
%%time
mass_ratio(mass)

NameError: name 'ratio' is not defined

In [ ]:
%%time
pool = rb.InterruptiblePool(processes=16)
mapping = pool.map(func= survival, iterable= tup_list)

In [ ]:
figure = np.reshape(mapping, [tup_num,tup_num])
plt.pcolormesh(e_b, a_p, figure.T, shading='auto')
plt.title('Mean Survival Times')
plt.xlabel('Binary Eccentricity (e)')
plt.ylabel('Planetary Semi-Major Axis (a)')
plt.xlim(0.0,0.7)
plt.ylim(1,5)

a_b = 2.278 + 3.824*e_b - 1.71*(e_b**2)
plt.plot(e_b, a_b, color='white')
plt.scatter(e_b, a_b, color='white')
plt.colorbar(label='Test Particle Survival Times')

In [ ]:
sa1 = rb.SimulationArchive('eb0.058_ap3.833_Np15.0_tup25.0.bin')
print("Number of snapshots: %d" % len(sa1))
print(sa1[1].t)

In [ ]:
# Functions to Plot Orbital evolution of Individual Planets

def orbit(archive):
    sim_arc = rb.SimulationArchive(archive)
    
    x_arr = []
    y_arr = []
    
    for snap in range(len(sim_arc)): 
        
        for i in range(2, sim_arc[snap].N):
            base = sim_arc[snap].particles[i]
        
            orb_element = base.a
            time = sim_arc[snap].t
            
            if orb_element not in y_arr:
                y_arr.append(orb_element)
            #if time not in x_arr:
                x_arr.append(time)
    
    fig = plt.figure(figsize=(50,20))
    plt.xlabel("Simulation Time", fontsize=35)
    plt.ylabel("Semi-Major Axis", fontsize=35)
    #plt.plot(x_arr,y_arr)
    plt.scatter(x_arr,y_arr, s=2, color='teal', marker=",")

In [ ]:
orbit('eb0.058_ap3.833_Np15.0_tup25.0.bin')

In [ ]:
file = open("SUNNY_map_tup25plan15.bin", "r")
test = file.readlines()
newlist = [line.rstrip("\n") for line in test]
mapper = np.reshape(newlist, [25,25])
print("Length of mapper:", len(mapper), mapper)
file.close()

In [ ]:
tup = 25
e_bin = np.linspace(0, 0.7, tup)
a_plan = np.linspace(1, 5, tup)
Nplan = 15
tuple_list = []

for e in e_bin:
    for a in a_plan:
        tuple_list.append((e,a,Nplan))
len(tuple_list)

In [ ]:
fig1 = plt.figure()
figure = np.array(mapper).astype(float)

im=plt.pcolormesh(e_bin, a_plan, figure.T, shading='auto', vmin=0, vmax=1e4*2*np.pi)

plt.title('Mean Survival Times')
plt.xlabel('Binary Eccentricity (e)')
plt.ylabel('Planetary Semi-Major Axis (a)')
plt.xlim(0.0,0.7)
plt.ylim(1,5)

a_bin = 2.278 + 3.824*e_bin - 1.71*(e_bin**2)
plt.plot(e_bin, a_bin, color='white')
plt.scatter(e_bin, a_bin, color='white')

plt.colorbar(im, label='Test Particle Survival Times')